In [184]:
import sys
import snowflake.connector as sf
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import stratifiedSample as ss

In [185]:
ctx =sf.connect(
    user='mgupta1@chewy.com',
    autocommit = 0,
    authenticator= "externalbrowser",
    account='chewy.us-east-1'
    )

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://chewy.okta.com/app/snowflake/exkmzig18yW58r3j00x7/sso/saml?SAMLRequest=lZLRbtowFIZfJfKuEztA1MwCKgZDRaKDlrRs3HnJIXgkdurjNNCnrwll6i5aaXeR8%2F32Z%2F%2Bnf30oC%2B8ZDEqtBiQMGPFApTqTKh%2BQh2Tqx8RDK1QmCq1gQI6A5HrYR1EWFR%2FVdqfu4akGtJ7bSCFvfwxIbRTXAiVyJUpAblO%2BGt3OeSdgvDLa6lQX5F3k84RABGOd4SWSoXR6O2srTmnTNEHTDbTJaYcxRtlX6qgT8uXCH9ydPuBDynon3hEOX765fZPq%2FASfaf0%2BQ8hvkmTpLxerhHiji%2BpYK6xLMCswzzKFh%2Fv5WQCdwfjm%2B%2FpXUKMPAq0fBqh0sy3EHlJdVrV1ewbui24ho4XOpbv2bDIg1V5mCY7v1o%2BzqLnbxKKyT0XczVU4KReLTZq%2FQCSm%2BzmLforbdS8l3uOl186p1xliDTN1atO6Jdbp%2BSzy2VUS9ngn5j0WhFG8Id7EtSmVsG3yopzuoDkGem9Fqyaqiv61pnDYly8yD%2BPjOopN9w9jhyuKqOmpWnKeFt4eb4b%2F9wZ9%2Bj77NnY%2FXBOzyVIXMj16U21KYT8uKgzCdkVm%2FrZFOZRCFqMsM4DoCisK3YwNCOum25oaCB2eT%2F13voev&RelayState=62

In [186]:
sql = """ 
SELECT * FROM MRCH_SANDBOX.LOYALTY_DS_TEMP_2_REVISED   
"""

cur= ctx.cursor()
cur.execute(sql)
data = cur.fetch_pandas_all()

In [187]:
data["MC_FLAG"] = 0
data.loc[data["PERC_CHEWY_SALES"] > 0  , "MC_FLAG" ] = 1

data["SW_FLAG"] = 0
data.loc[data["SW_PROMO_FLAG"].isin(['SINGLE_PROMO' , 'MULTI_PROMO'])  , "SW_FLAG" ] = 1

data['TENURE_MEAN'] = data.groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT'])['TENURE'].transform('mean')


# data.loc[data["PERC_CHEWY_SALES"] > 95 , "PERC_CHEWY_SALES_CAT" ] = 'Greater than 95'


# data.loc[data["PERC_CHEWY_SALES"] <=50 , "PERC_CHEWY_SALES_CAT" ] = 'Less than 50'
# data.loc[(data["PERC_CHEWY_SALES"] >50 ) & (data["PERC_CHEWY_SALES"] <= 90), "PERC_CHEWY_SALES_CAT" ] = 'Between 50 and 90'
# data.loc[data["PERC_CHEWY_SALES"] > 90 , "PERC_CHEWY_SALES_CAT" ] = 'Greater than 90'


# for columns in ['AS_FLAG', 'PET_FLAG','SW_FLAG','MC_FLAG']:
#     data[columns] = data[columns].astype("object")


In [188]:
data["TENURE_BUCKETS" ] = 'TENURE_2'
data.loc[data["TENURE"] <= data['TENURE_MEAN'] , "TENURE_BUCKETS" ] = 'TENURE_1'


data["concat"] = data['AS_FLAG'].astype(str) + '_'  + data['PET_FLAG'].astype(str) + '_'  + data['MC_FLAG'].astype(str)


# data["concat"] = data['AS_FLAG'].astype(str) + '_' +  data['TENURE_BUCKETS'].astype(str) + '_' + data['PET_FLAG'].astype(str) + '_' +data['MC2_FLAG'].astype(str) + '_' +data['SW_FLAG'].astype(str) + '_' + data['MC_FLAG'].astype(str)


# data.loc[(data["TENURE"] >1500 ) & (data["TENURE"] <= 2100) , "TENURE_BUCKETS" ] = 'TENURE_3'
# data.loc[(data["TENURE"] >2100 )  , "TENURE_BUCKETS" ] = 'TENURE_4'

In [189]:
data['COUNT'] = data.groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT'])['CUSTOMER_ID'].transform('count')
data["combo_COUNT"] = data.groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT' , 'concat'])['CUSTOMER_ID'].transform('nunique')
# data["CUSTOMER_PROP"] = data['COUNT'] / data.shape[0]

data.loc[data["COHORT"] == 'COHORT_1' , "CUSTOMER_REQD"] = data["CUSTOMER_PROP"] * 12000
data.loc[data["COHORT"] == 'COHORT_2' , "CUSTOMER_REQD"] = data["CUSTOMER_PROP"] * 8000


In [191]:
filtered_Data = data[data["combo_COUNT"] >=3]

In [192]:
filtered_Data

,REGULAR_CUSTOMER,COHORT,QUARTILE,CUSTOMER_ID,AS_FLAG,TENURE,PET_FLAG,MC2_FLAG,SW_PROMO_FLAG,PERC_CHEWY_SALES,MC_FLAG,SW_FLAG,TENURE_MEAN,TENURE_BUCKETS,concat,COUNT,combo_COUNT
0,0,COHORT_1,3,190876909,0,873,SINGLE_PET,0,NO_PROMO,NaN,0,0,1476.131257,TENURE_1,0_SINGLE_PET_0,43769,3596
1,0,COHORT_1,3,280089165,0,420,NO_PET,0,NO_PROMO,NaN,0,0,1476.131257,TENURE_1,0_NO_PET_0,43769,4176
2,0,COHORT_1,1,909880089,0,394,NO_PET,0,NO_PROMO,100.0,1,0,1572.788127,TENURE_1,0_NO_PET_1,43781,1519
3,0,COHORT_2,1,73512166,0,1643,NO_PET,0,NO_PROMO,NaN,0,0,1680.145522,TENURE_1,0_NO_PET_0,2144,137
4,0,COHORT_1,1,93864599,1,1448,SINGLE_PET,0,NO_PROMO,NaN,0,0,1572.788127,TENURE_1,1_SINGLE_PET_0,43781,7213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323502,0,COHORT_1,3,51396101,1,1908,NO_PET,0,NO_PROMO,100.0,1,0,1476.131257,TENURE_2,1_NO_PET_1,43769,2321
323503,0,COHORT_1,2,901003122,1,372,NO_PET,1,NO_PROMO,NaN,0,0,1507.676696,TENURE_1,1_NO_PET_0,43739,7481
323504,1,COHORT_2,3,136926473,0,966,SINGLE_PET,0,NO_PROMO,NaN,0,0,1753.258639,TENURE_1,0_SINGLE_PET_0,14325,2316
323505,1,COHORT_2,2,106532515,0,1335,MULTI_PET,0,NO_PROMO,NaN,0,0,1752.076397,TENURE_1,0_MULTI_PET_0,14320,1819


In [ ]:
selected_data = pd.DataFrame()
for cohort in filtered_Data["COHORT"].unique():
    for regular in filtered_Data["REGULAR_CUSTOMER"].unique():
        for quartile in filtered_Data["QUARTILE"].unique():
            data_sel = filtered_Data[(data["REGULAR_CUSTOMER"] == regular) & (filtered_Data["COHORT"] == cohort) & (filtered_Data["QUARTILE"] == quartile)]

            perc = data_sel["CUSTOMER_REQD"].min() * 2 /data_sel.shape[0]
            X_train, Test = train_test_split(data_sel , test_size= round(perc, 4) , stratify = data_sel['concat'])
            # Test["Flag"] = "Test" 

            # perc = data_sel["CUSTOMER_REQD"].min()/X_train.shape[0]
            # X_train_fil = X_train[X_train["combo_COUNT"]>= check_count ]
            Test_new, Control = train_test_split(Test , test_size= round(perc, 4) , stratify = Test['concat'])
            Control["Flag"] = "Control"   
            Test_new["Flag"] = "Test" 

            temp_df = pd.concat([Test[["CUSTOMER_ID" , 'Flag']] ,  Control[["CUSTOMER_ID" , 'Flag']]  ])   
            selected_data = pd.concat([selected_data, temp_df])

        #     Test, Control = train_test_split(X_test , test_size= 0.5, stratify= X_test[['AS_FLAG',
        #    'TENURE_BUCKETS', 'PET_FLAG', 'MC2_FLAG', 'SW_PROMO_FLAG']] )
              



In [177]:
data_merge = data.merge(selected_data , on = "CUSTOMER_ID" , how = 'left')

In [178]:
data_merge["Flag"].fillna("Not Assigned yet" , inplace = True)

In [151]:
data.columns

Index(['REGULAR_CUSTOMER', 'COHORT', 'QUARTILE', 'CUSTOMER_ID', 'AS_FLAG',
       'TENURE', 'PET_FLAG', 'MC2_FLAG', 'SW_PROMO_FLAG', 'PERC_CHEWY_SALES',
       'MC_FLAG', 'SW_FLAG', 'TENURE_MEAN', 'TENURE_BUCKETS', 'concat',
       'COUNT', 'combo_COUNT'],
      dtype='object')

In [180]:
test_data = data_merge[data_merge["Flag"]!= "Not Assigned yet"].groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT', 'Flag' , "PET_FLAG" , "MC_FLAG" , "AS_FLAG"])["CUSTOMER_ID"].nunique().reset_index()
# pivot_data = test_data.pivot(index= ['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT' , 'Flag'] ,
#                              columns = ["PET_FLAG" ],
#                               values = "CUSTOMER_ID" )
# pivot_data.reset_index()

test_data.to_csv("check.csv")

In [179]:
data_merge[data_merge["Flag"]!= "Not Assigned yet"].groupby(["Flag"])["CUSTOMER_ID"].count()

Flag
Control    21087
Test       21089
Name: CUSTOMER_ID, dtype: int64

In [183]:
data_merge.to_csv("wave1.csv" , index =  False)

In [150]:
test  = data_merge[data_merge["Flag"]!= "Not Assigned yet"].groupby(['REGULAR_CUSTOMER' , 'QUARTILE' , 'COHORT', 'Flag']).agg({"CUSTOMER_ID" : 'count' , 'MC_FLAG' : 'sum' , 
                                                                                                                               'TENURE' : 'mean' , 'AS_FLAG' : 'sum'}).reset_index()
test["AS_PERC"] = test["AS_FLAG"] * 100 / test["CUSTOMER_ID"]
test["MC2_FLAG_Perc"] = test["MC2_FLAG"] * 100 / test["CUSTOMER_ID"]
pivot_data
# ["CUSTOMER_ID"].count()

,REGULAR_CUSTOMER,QUARTILE,COHORT,Flag,CUSTOMER_ID,MC2_FLAG,TENURE,AS_FLAG,AS_PERC,MC2_FLAG_Perc
0,0,1,COHORT_1,Control,1720,411,1593.012791,852,49.534884,23.895349
1,0,1,COHORT_1,Test,1720,386,1583.475000,852,49.534884,22.441860
2,0,1,COHORT_2,Control,144,22,1660.368056,108,75.000000,15.277778
3,0,1,COHORT_2,Test,1000,162,1703.733000,753,75.300000,16.200000
4,0,2,COHORT_1,Control,1720,535,1485.036047,1077,62.616279,31.104651
5,0,2,COHORT_1,Test,1720,496,1486.447674,1078,62.674419,28.837209
6,0,2,COHORT_2,Control,145,23,1681.186207,118,81.379310,15.862069
7,0,2,COHORT_2,Test,1000,189,1610.010000,821,82.100000,18.900000
8,0,3,COHORT_1,Control,1720,590,1439.982558,1250,72.674419,34.302326
9,0,3,COHORT_1,Test,1720,561,1481.706977,1249,72.616279,32.616279


In [171]:
selected_data = pd.DataFrame()

for regular in filtered_Data["REGULAR_CUSTOMER"].unique():
    for quartile in filtered_Data["QUARTILE"].unique():
        data_sel = filtered_Data[(data["REGULAR_CUSTOMER"] == regular) & (filtered_Data["COHORT"] == "COHORT_1") & (filtered_Data["QUARTILE"] == quartile)]
 
        perc = round(2200/data_sel.shape[0],2)
       
        X_train, Test = train_test_split(data_sel , test_size= 1600 ,  stratify = data_sel['concat'])
        Test["Flag"] = "Test"

        perc = round(2200/X_train.shape[0], 2)
 
        if(perc>0.50):
           
            Control , last_split = train_test_split(X_train , test_size = 1600 , stratify = X_train['concat'])
            Control["Flag"] = "Control"   
        else:
            last_split , Control   = train_test_split(X_train , test_size= 1600 , stratify = X_train['concat'])
            Control["Flag"] = "Control"  
 

        temp_df = pd.concat([Test[["CUSTOMER_ID" , 'Flag']] ,  Control[["CUSTOMER_ID" , 'Flag']]  ])   
        selected_data = pd.concat([selected_data, temp_df])

    #     Test, Control = train_test_split(X_test , test_size= 0.5, stratify= X_test[['AS_FLAG',
    #    'TENURE_BUCKETS', 'PET_FLAG', 'MC2_FLAG', 'SW_PROMO_FLAG']] )
            

In [172]:
selected_data.shape

(25600, 2)

In [173]:
# selected_data = pd.DataFrame()
for regular in  [1]:
    for quartile in filtered_Data["QUARTILE"].unique():

        data_sel = filtered_Data[(filtered_Data["REGULAR_CUSTOMER"] == regular) & (filtered_Data["COHORT"] == "COHORT_2") & (filtered_Data["QUARTILE"] == quartile)]

        perc = round(1000/data_sel.shape[0],2)
 
        X_train, Test = train_test_split(data_sel , test_size= 1000 ,  stratify = data_sel['concat'])
        Test["Flag"] = "Test"
        # X_train.groupby(['concat'])["CUSTOMER_ID"].count().reset_index()["CUSTOMER_ID"].min()
        # if( == 1)

        perc = round(1000/X_train.shape[0], 2)
        # count = 
 
        if(perc>0.50):
           
            Control , last_split = train_test_split(X_train , test_size = 1000 , stratify = X_train['concat'])
            Control["Flag"] = "Control"   
        else:
            last_split , Control   = train_test_split(X_train , test_size= 1000 , stratify = X_train['concat'])
            Control["Flag"] = "Control"  


        temp_df = pd.concat([Test[["CUSTOMER_ID" , 'Flag']] ,  Control[["CUSTOMER_ID" , 'Flag']]  ])   
        selected_data = pd.concat([selected_data, temp_df])

In [176]:
selected_data.shape

(42176, 2)

In [175]:
for regular in  [0]:
    for quartile in filtered_Data["QUARTILE"].unique():

        data_sel = filtered_Data[(filtered_Data["REGULAR_CUSTOMER"] == regular) & (filtered_Data["COHORT"] == "COHORT_2") & (filtered_Data["QUARTILE"] == quartile)]

        perc = round(1000/data_sel.shape[0],2)
 
        Control, Test = train_test_split(data_sel , test_size= 0.5 ,  stratify = data_sel['concat'])
        Test["Flag"] = "Test"
        Control["Flag"] = "Control"  
        # # X_train.groupby(['concat'])["CUSTOMER_ID"].count().reset_index()["CUSTOMER_ID"].min()
        # # if( == 1)

        # perc = round(1000/X_train.shape[0], 2)
        # # count = 
 
        # if(perc>0.50):
           
        #     Control , last_split = train_test_split(X_train , test_size = 1000 , stratify = X_train['concat'])
        #     Control["Flag"] = "Control"   
        # else:
        #     last_split , Control   = train_test_split(X_train , test_size= 1000 , stratify = X_train['concat'])
        #     Control["Flag"] = "Control"  


        temp_df = pd.concat([Test[["CUSTOMER_ID" , 'Flag']] ,  Control[["CUSTOMER_ID" , 'Flag']]  ])   
        selected_data = pd.concat([selected_data, temp_df])

In [29]:
X_train.sort_values(by = "combo_COUNT")

,REGULAR_CUSTOMER,COHORT,QUARTILE,CUSTOMER_ID,AS_FLAG,TENURE,PET_FLAG,MC2_FLAG,SW_PROMO_FLAG,PERC_CHEWY_SALES,MC_FLAG,SW_FLAG,TENURE_MEAN,TENURE_BUCKETS,concat,COUNT,combo_COUNT,CUSTOMER_PROP,CUSTOMER_REQD
171535,0,COHORT_2,1,130282630,0,1186,MULTI_PET,0,SINGLE_PROMO,100.0,1,1,1680.145522,TENURE_1,0_TENURE_1_MULTI_PET_0_1_1,2144,3,0.006627,53.018945
246328,0,COHORT_2,1,200298772,0,837,NO_PET,1,NO_PROMO,100.0,1,0,1680.145522,TENURE_1,0_TENURE_1_NO_PET_1_0_1,2144,3,0.006627,53.018945
2022,0,COHORT_2,1,56960632,0,1841,SINGLE_PET,1,NO_PROMO,NaN,0,0,1680.145522,TENURE_2,0_TENURE_2_SINGLE_PET_1_0_0,2144,3,0.006627,53.018945
303278,0,COHORT_2,1,41718427,0,2041,SINGLE_PET,0,MULTI_PROMO,100.0,1,1,1680.145522,TENURE_2,0_TENURE_2_SINGLE_PET_0_1_1,2144,3,0.006627,53.018945
249070,0,COHORT_2,1,886222253,0,709,MULTI_PET,0,SINGLE_PROMO,100.0,1,1,1680.145522,TENURE_1,0_TENURE_1_MULTI_PET_0_1_1,2144,3,0.006627,53.018945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131960,0,COHORT_2,1,15300652,1,2574,MULTI_PET,0,NO_PROMO,NaN,0,0,1680.145522,TENURE_2,1_TENURE_2_MULTI_PET_0_0_0,2144,224,0.006627,53.018945
94866,0,COHORT_2,1,42773202,1,1748,MULTI_PET,0,NO_PROMO,NaN,0,0,1680.145522,TENURE_2,1_TENURE_2_MULTI_PET_0_0_0,2144,224,0.006627,53.018945
299774,0,COHORT_2,1,45851002,1,1976,MULTI_PET,0,NO_PROMO,NaN,0,0,1680.145522,TENURE_2,1_TENURE_2_MULTI_PET_0_0_0,2144,224,0.006627,53.018945
116958,0,COHORT_2,1,52770941,1,1892,MULTI_PET,0,NO_PROMO,NaN,0,0,1680.145522,TENURE_2,1_TENURE_2_MULTI_PET_0_0_0,2144,224,0.006627,53.018945


In [83]:
df = pd.DataFrame(data = [ ['a' , 1]  , ['a' , 2], ['a' , 3]] , columns = ["val1" , "val2"] )

In [103]:
train , test  = train_test_split(df , train_size = 1,  stratify= df["val1"])

In [104]:
test

,val1,val2
2,a,3
1,a,2


In [102]:
train

,val1,val2
1,a,2
2,a,3
